In [1]:
!pip install gensim

In [2]:
import re  
import pandas as pd 
from time import time  
from collections import defaultdict 
import string 
import multiprocessing
import os
import gensim
import sklearn
from sklearn import linear_model
from collections import Counter
import numpy as np
import scipy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report
from nltk.tokenize import word_tokenize

# word2vec
from gensim.models import Word2Vec, KeyedVectors, FastText
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
from collections import Counter

punctuation = string.punctuation + "«»“”‘’…—"

stopwords_spanish = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt'
).values
stopwords_spanish = Counter(stopwords_spanish.flatten().tolist())

stopwords_english = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords_english = Counter(stopwords_english.flatten().tolist())

def simple_tokenizer(doc, stopwords, lower=False):
    if lower:
        tokenized_doc = doc.translate(str.maketrans(
            '', '', punctuation)).lower().split()

    tokenized_doc = doc.translate(str.maketrans('', '', punctuation)).split()

    tokenized_doc = [
        token for token in tokenized_doc if token.lower() not in stopwords
    ]
    return tokenized_doc

In [5]:
import pickle

path =  "../../Data/train/df_us_train.pickle"
df_us_train = pickle.load(open(path, "rb"))

In [10]:
us_emoji_w2v = Word2Vec(min_count=200,
                      window=4,
                      vector_size=200,
                      sample=6e-5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      negative=20,
                      workers=multiprocessing.cpu_count())

content = df_us_train["text"]
cleaned_content = [simple_tokenizer(doc, stopwords=stopwords_english) for doc in content.values]
phrases = Phrases(cleaned_content, min_count=150, progress_per=5000) 
bigram = Phraser(phrases)
sentences = bigram[cleaned_content]

us_emoji_w2v.build_vocab(sentences, progress_per=10000)

t = time()
us_emoji_w2v.train(sentences, total_examples=us_emoji_w2v.corpus_count, epochs=15, report_delay=10)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

us_emoji_w2v.init_sims(replace=True)

2022-06-15 16:58:30,081 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=200, alpha=0.03>', 'datetime': '2022-06-15T16:58:30.081955', 'gensim': '4.2.0', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
2022-06-15 16:58:34,982 : INFO : collecting all words and their counts
2022-06-15 16:58:34,982 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2022-06-15 16:58:35,027 : INFO : PROGRESS: at sentence #5000, processed 29821 words and 36340 word types
2022-06-15 16:58:35,069 : INFO : PROGRESS: at sentence #10000, processed 59723 words and 67133 word types
2022-06-15 16:58:35,129 : INFO : PROGRESS: at sentence #15000, processed 89629 words and 96385 word types
2022-06-15 16:58:35,182 : INFO : PROGRESS: at sentence #20000, processed 119889 words and 124696 word types
2022-06-15 16:58:35,255 : INFO : PROGRESS: at sentence #25000, processed 149690 words an

2022-06-15 16:58:39,163 : INFO : PROGRESS: at sentence #360000, processed 2153424 words and 1545014 word types
2022-06-15 16:58:39,215 : INFO : PROGRESS: at sentence #365000, processed 2183479 words and 1563543 word types
2022-06-15 16:58:39,260 : INFO : PROGRESS: at sentence #370000, processed 2213300 words and 1581910 word types
2022-06-15 16:58:39,311 : INFO : PROGRESS: at sentence #375000, processed 2243347 words and 1600485 word types
2022-06-15 16:58:39,365 : INFO : PROGRESS: at sentence #380000, processed 2273268 words and 1619180 word types
2022-06-15 16:58:39,416 : INFO : PROGRESS: at sentence #385000, processed 2303285 words and 1637550 word types
2022-06-15 16:58:39,440 : INFO : collected 1645907 token types (unigram + bigrams) from a corpus of 2317037 words and 387292 sentences
2022-06-15 16:58:39,440 : INFO : merged Phrases<1645907 vocab, min_count=150, threshold=10.0, max_vocab_size=40000000>
2022-06-15 16:58:39,440 : INFO : Phrases lifecycle event {'msg': 'built Phrases<

2022-06-15 16:58:47,463 : INFO : Word2Vec lifecycle event {'msg': 'training model with 8 workers on 1509 vocabulary and 200 features, using sg=0 hs=0 sample=6e-05 negative=20 window=4 shrink_windows=True', 'datetime': '2022-06-15T16:58:47.463561', 'gensim': '4.2.0', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'train'}
2022-06-15 16:58:48,481 : INFO : EPOCH 0 - PROGRESS: at 9.70% examples, 37285 words/s, in_qsize 14, out_qsize 1
2022-06-15 16:58:52,343 : INFO : EPOCH 0: training on 2262016 raw words (386809 effective words) took 4.9s, 79462 effective words/s
2022-06-15 16:58:53,567 : INFO : EPOCH 1 - PROGRESS: at 14.13% examples, 45103 words/s, in_qsize 13, out_qsize 1
2022-06-15 16:58:56,917 : INFO : EPOCH 1: training on 2262016 raw words (387200 effective words) took 4.6s, 84859 effective words/s
2022-06-15 16:58:57,976 : INFO : EPOCH 2 - PROGRESS: at 11.03% examples, 41992 words/s, in_qsize 13, out

Time to train the model: 1.18 mins


In [25]:
us_emoji_w2v.wv.key_to_index

{'user': 0,
 'love': 1,
 'amp': 2,
 'day': 3,
 'night': 4,
 'time': 5,
 'Park': 6,
 'Love': 7,
 'University': 8,
 'California': 9,
 'beautiful': 10,
 'Happy': 11,
 'Beach': 12,
 'happy': 13,
 'fun': 14,
 'Center': 15,
 'Christmas': 16,
 'family': 17,
 'favorite': 18,
 'City': 19,
 'life': 20,
 'friends': 21,
 'tonight': 22,
 'weekend': 23,
 'amazing': 24,
 'girl': 25,
 'morning': 26,
 'School': 27,
 'Lake': 28,
 'York': 29,
 'friend': 30,
 'Day': 31,
 'NYC': 32,
 '2': 33,
 'people': 34,
 'Florida': 35,
 'Texas': 36,
 'birthday': 37,
 'girls': 38,
 'week': 39,
 'Los_Angeles': 40,
 'baby': 41,
 'days': 42,
 'Repost': 43,
 'Miami': 44,
 'heart': 45,
 'Chicago': 46,
 'Stadium': 47,
 'House': 48,
 'wait': 49,
 'Downtown': 50,
 'York_York': 51,
 'pretty': 52,
 'lol': 53,
 'Island': 54,
 'ready': 55,
 'picture': 56,
 '2016': 57,
 'game': 58,
 'guys': 59,
 'tbt': 60,
 'nyc': 61,
 'Hotel': 62,
 'Bar': 63,
 'view': 64,
 'Sunday': 65,
 '1': 66,
 'guy': 67,
 'summer': 68,
 'beach': 69,
 'College':

In [28]:
us_emoji_w2v.wv.most_similar(positive=["Christmas"])

[('christmas', 0.7948068380355835),
 ('happyholidays', 0.7622090578079224),
 ('christmastree', 0.7594813108444214),
 ('Xmas', 0.745563805103302),
 ('merrychristmas', 0.7280957102775574),
 ('lot_Christmas', 0.7279301285743713),
 ('holidays', 0.7263860702514648),
 ('Christmas_tree', 0.7261241674423218),
 ('merry', 0.7209651470184326),
 ('Merry', 0.7163508534431458)]

In [30]:
us_emoji_w2v.wv.most_similar(positive=["lol"])

[('funny', 0.8770618438720703),
 ('lmao', 0.8752225041389465),
 ('Lol', 0.8519217371940613),
 ('haha', 0.8450767397880554),
 ('hilarious', 0.8235787153244019),
 ('told', 0.8166170120239258),
 ('dead', 0.8078774213790894),
 ('tho', 0.8038719892501831),
 ('mad', 0.7963632345199585),
 ('swear', 0.7932891249656677)]

In [35]:
us_emoji_w2v.wv.most_similar(positive=["nyc"])

[('newyork', 0.9547122716903687),
 ('ny', 0.9420251846313477),
 ('manhattan', 0.9400611519813538),
 ('newyorkcity', 0.9347207546234131),
 ('brooklyn', 0.8718341588973999),
 ('NewYork', 0.8151965737342834),
 ('Empire_Building', 0.8134568929672241),
 ('Manhattan_York', 0.812926709651947),
 ('Brooklyn_Bridge', 0.8073521256446838),
 ('NYC', 0.8068370819091797)]

In [118]:
def sentence_embedding(doc, w2v):
    pre_tokens = simple_tokenizer(doc, stopwords_english, lower=False)
    sentence_vect = np.zeros(w2v.vector_size)
    m = 0
    for token in pre_tokens:
        if token in w2v.wv:
            sentence_vect += w2v.wv[token]
            m += 1
    sentence_vect *= 1/m if m>0 else 1
    return sentence_vect

In [119]:
text = 'Selfies for summatime @ Drexel University '
sentence_embedding(text, us_emoji_w2v)

array([ 0.05258413, -0.05869599,  0.07374898,  0.05271254,  0.10018036,
        0.10197563,  0.04695616,  0.08050372,  0.11423416,  0.06558054,
       -0.01431429, -0.0670277 ,  0.00197182,  0.0047567 , -0.05848593,
       -0.05764413, -0.08735098, -0.04672073,  0.05703964, -0.0642032 ,
        0.06155309, -0.05893533, -0.0165066 ,  0.01464991,  0.05360454,
        0.02000251,  0.08777064, -0.03402002, -0.03354172,  0.0610544 ,
        0.06087575, -0.00920816,  0.09910131, -0.04011815,  0.17804979,
       -0.06459375, -0.01501393, -0.02038939,  0.00767087,  0.00352707,
        0.01339428,  0.13464119, -0.04553435,  0.07318062,  0.12162764,
       -0.04713735,  0.03409766, -0.09665535,  0.03297074, -0.02260041,
        0.06641608, -0.07184253,  0.05569032, -0.03323142,  0.00063829,
       -0.02382268,  0.03002416, -0.108971  , -0.18713465, -0.05489816,
       -0.06531326,  0.05204288, -0.05940802,  0.0856853 , -0.05804159,
        0.05194528, -0.14928423, -0.05097568,  0.05409653, -0.03

In [57]:
us_emoji_w2v.save("us_emoji_w2v.model")

2022-06-15 17:40:46,975 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'us_emoji_w2v.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-06-15T17:40:46.975360', 'gensim': '4.2.0', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'saving'}
2022-06-15 17:40:46,977 : INFO : not storing attribute cum_table
2022-06-15 17:40:46,987 : INFO : saved us_emoji_w2v.model


In [137]:
%%time
us_train_sentence_embedding = pd.DataFrame(np.array(
    [ sentence_embedding(doc, us_emoji_w2v) for doc in df_us_train["text"]]
))
us_train_sentence_embedding["id"] = df_us_train["id"]

Wall time: 26.4 s


In [143]:
us_train_sentence_embedding["label"] = df_us_train["label"]

In [151]:
X = us_train_sentence_embedding.drop(columns=["label", "id"])

In [149]:
n_labels = us_train_sentence_embedding["label"].unique().shape[0]

In [148]:
from sklearn.cluster import KMeans

In [152]:
kmeans = KMeans(n_clusters=n_labels, random_state=0).fit(X)

In [153]:
us_train_sentence_embedding["kmeans"] = kmeans.labels_

In [154]:
from sklearn.metrics.cluster import v_measure_score

In [155]:
v_measure_score(us_train_sentence_embedding["label"], us_train_sentence_embedding["kmeans"])

0.04851481180375604

In [156]:
from sklearn.metrics.cluster import rand_score

In [157]:
rand_score(us_train_sentence_embedding["label"], us_train_sentence_embedding["kmeans"])

0.8648050610680522

In [169]:
df_us_train[
    us_train_sentence_embedding["kmeans"] == 5
]["text"]

28        Enjoying Christmas in shorts and tee's.. I lov...
44        the last pick of the season and tree is buddin...
210       || Rancho Santa Fe is where you want to be. Do...
216       @ Pipe Creek Christmas Tree Farm and Pumpkin P...
415       Happy December 1st! We're having a great time ...
                                ...                        
387212    Guerrier Christmas 5 years later love my famil...
387230    Merry #Christmas &amp; Happy Holidays! # @ Sno...
387250    Merry Christmas. #alwaysaboutlove #mygirl #chr...
387264    'Tis the season of pure happiness @ Christmas ...
387277    This sweet little elf (and his little reindeer...
Name: text, Length: 10554, dtype: object